<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/gridVeg_plant_abundance_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Security

* The user must load a `json` file containing the BigQuery API key into the local directory `/content/...`
* The user must have a Google Maps API key to enable mapping. 
   * CAUTION make sure the key is deleted from the current instance of the notebook before sharing

# Tools

* Remember that the file containing authorization keys for Big Query must be loaded into the virutual envrionment manually.

In [ ]:
install.packages("bigrquery")
library(bigrquery)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(tidyverse)

# Source

## Database Connection

In [ ]:
# BigQuery API Key
bq_auth(path = "/content/mpg-data-warehouse-api_key-master.json")

In [ ]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [ ]:
billing <- bq_test_project()

### Survey Effort

In [ ]:
con_survey_effort <- dbConnect(
  bigrquery::bigquery(),
  project = "mpg-data-warehouse",
  dataset = "vegetation_gridVeg_summaries",
  billing = billing
)

In [ ]:
dbListTables(con_survey_effort)

[1] "gridVeg_groundCover_intercepts" "gridVeg_plant_intercepts"      
[3] "gridVeg_species_richness"       "gridVeg_survey_effort"

In [ ]:
intercepts_sql <- "SELECT survey_ID, grid_point, key_plant_code, intercepts_pct
                   FROM `mpg-data-warehouse.vegetation_gridVeg_summaries.gridVeg_plant_intercepts`"

In [ ]:
bq_intercepts <- bq_project_query(billing, intercepts_sql)

In [ ]:
tb_intercepts <- bq_table_download(bq_intercepts)

In [ ]:
df_intercepts <- as.data.frame(tb_intercepts)

In [ ]:
glimpse(df_intercepts)

Rows: 25,089
Columns: 4
$ survey_ID      <chr> "436", "436", "436", "436", "436", "436", "436", "436"…
$ grid_point     <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, …
$ key_plant_code <chr> "HEUCYL", "ALLCER", "GEUTRI", "ERIG_SP", "ARESER", "GA…
$ intercepts_pct <dbl> 2.5, 0.5, 1.0, 0.5, 1.0, 1.5, 1.0, 2.5, 0.5, 1.0, 14.5…


### Survey Metadata

In [ ]:
con_survey_meta <- dbConnect(
  bigrquery::bigquery(),
  project = "mpg-data-warehouse",
  dataset = "vegetation_point_intercept_gridVeg",
  billing = billing
)

In [ ]:
dbListTables(con_survey_meta)

[1] "gridVeg_additional_species"         "gridVeg_ground_cover_metadata"     
[3] "gridVeg_image_metadata"             "gridVeg_point_intercept_ground"    
[5] "gridVeg_point_intercept_vegetation" "gridVeg_shrub_tree"                
[7] "gridVeg_survey_metadata"

In [ ]:
meta_sql <- "SELECT survey_ID, year, survey_sequence FROM `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_survey_metadata`"

In [ ]:
bq_meta <- bq_project_query(billing, meta_sql)

In [ ]:
tb_meta <- bq_table_download(bq_meta)

In [ ]:
df_meta <- as.data.frame(tb_meta)

In [ ]:
glimpse(df_meta)

Rows: 1,472
Columns: 3
$ survey_ID       <chr> "F31C56A8-912D-410C-A17D-4C2DD75F71A4", "A19E87E6-A89…
$ year            <int> 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016,…
$ survey_sequence <chr> "2016", "2016", "2016", "2016", "2016", "2016", "2016…


### Position Classification

In [ ]:
con_position_class <- dbConnect(
  bigrquery::bigquery(),
  project = "mpg-data-warehouse",
  dataset = "grid_point_summaries",
  billing = billing
)

In [ ]:
dbListTables(con_position_class)

[1] "location"                         "location_position"               
[3] "location_position_classification"

In [ ]:
position_sql <- "SELECT 
                grid_point, aspect_mean_deg, elevation_mean_m, slope_mean_deg, 
                cover_type_2016_gridVeg, type3_vegetation_indicators, type4_indicators_history
             FROM `mpg-data-warehouse.grid_point_summaries.location_position_classification`"

In [ ]:
bq_position <- bq_project_query(billing, position_sql)

In [ ]:
tb_position <- bq_table_download(bq_position)

In [ ]:
df_position <- as.data.frame(tb_position)

In [ ]:
glimpse(df_position)

Rows: 582
Columns: 7
$ grid_point                  <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13…
$ aspect_mean_deg             <dbl> 334.7050, 45.3030, 221.3340, 290.4890, 28…
$ elevation_mean_m            <dbl> 1395.64, 1456.09, 1126.90, 1166.33, 1179.…
$ slope_mean_deg              <dbl> 28.44230, 12.22630, 4.25130, 2.68361, 4.2…
$ cover_type_2016_gridVeg     <chr> "woodland/forest", "non-irrigated grassla…
$ type3_vegetation_indicators <chr> "mixed canopy conifer", "uncultivated gra…
$ type4_indicators_history    <chr> "mixed canopy conifer", "uncultivated gra…


# Wrangle

## Reshape Intercepts/Effort

In [ ]:
df_intercepts <- df_intercepts %>%
  spread(key_plant_code, intercepts_pct, fill = 0)

## Join Tables

In [ ]:
df_abundance_matrix <- df_intercepts %>%
  left_join(df_meta)

Joining, by = "survey_ID"



In [ ]:
df_abundance_matrix <- df_abundance_matrix %>%
  left_join(df_position)

Joining, by = "grid_point"



## Reorder Columns

In [ ]:
head(df_abundance_matrix, n=4)

,survey_ID,grid_point,ABIGRA,ABILAS,ACEGLA,ACHMIL,ACTRUB,AGAURT,AGOAUR,AGOGLA,⋯,XERTEN,ZEAMAY,year,survey_sequence,aspect_mean_deg,elevation_mean_m,slope_mean_deg,cover_type_2016_gridVeg,type3_vegetation_indicators,type4_indicators_history
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,012C5FAD-2451-41B0-9E2F-432D1ECEB55C,285,0,0,0,0.0,0,0,0,0,⋯,0,0,2016,2016,138.749,1352.10,24.51410,shrubland,mixed sage and bitterbrush,mixed sage and bitterbrush
2,0133805F-8237-4190-B125-14E883158664,505,0,0,8,0.0,0,0,0,0,⋯,0,0,2016,2016,319.161,1194.89,16.50500,woody draw,wooded draw,wooded draw
3,0155DA4F-B744-4B92-BAE3-326ABC3C586F,401,0,0,0,6.5,0,0,0,0,⋯,0,0,2017,2017,178.871,1588.60,8.31327,woodland/forest,open canopy conifer,open canopy conifer
4,0159CEF8-3C0B-42D1-B961-03DA47A2C274,23,0,0,0,0.0,0,0,0,0,⋯,0,0,2016,2016,264.245,1224.44,9.72424,shrubland,bitterbrush,bitterbrush


In [ ]:
df_abundance_matrix <- df_abundance_matrix[,c(1,492,493,2,494:ncol(df_abundance_matrix),3:491)]

In [ ]:
head(df_abundance_matrix, n=4)

,survey_ID,year,survey_sequence,grid_point,aspect_mean_deg,elevation_mean_m,slope_mean_deg,cover_type_2016_gridVeg,type3_vegetation_indicators,type4_indicators_history,⋯,VERVIR,VICAME,VICSAT,VIOCAN,VIOL_SP,VIONUT,VULOCT,WOOORE,XERTEN,ZEAMAY
,<chr>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,012C5FAD-2451-41B0-9E2F-432D1ECEB55C,2016,2016,285,138.749,1352.10,24.51410,shrubland,mixed sage and bitterbrush,mixed sage and bitterbrush,⋯,0,0,0,0,0,0,0,0,0,0
2,0133805F-8237-4190-B125-14E883158664,2016,2016,505,319.161,1194.89,16.50500,woody draw,wooded draw,wooded draw,⋯,0,0,0,0,0,0,0,0,0,0
3,0155DA4F-B744-4B92-BAE3-326ABC3C586F,2017,2017,401,178.871,1588.60,8.31327,woodland/forest,open canopy conifer,open canopy conifer,⋯,0,0,0,0,0,0,0,0,0,0
4,0159CEF8-3C0B-42D1-B961-03DA47A2C274,2016,2016,23,264.245,1224.44,9.72424,shrubland,bitterbrush,bitterbrush,⋯,0,0,0,0,0,0,0,0,0,0


# Output

In [ ]:
write_csv(df_abundance_matrix, path = "gridVeg_plant_abundance_matrix_WRANGLE.csv")